# Analyse the resulting tables and concatenate them together 

In [ ]:
import pandas as pd
#import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
from matplotlib import pyplot as plt
from numpy import nan
import ast

## 1. Large tables :

ClinVar 1st stage of mapping (to Ensembl using rs identifiers)

ClinVar 2nd stage of mapping (to Ensembl using coordinates)

ClinVar 3rd - fetched SNVs (the variants for which the molecular consequences have been fetched) <- ready to concat

ClinVar indels edded unfiltered (actually this contains some SNPs as well, no reason identified) <- ready to concat

Rafique 1st stage of annotatopn (with Vcfanno)

Rafique 2nd stage of annotatopn (with PubMed)

Rafique manual annotation

In [ ]:
#ClinVar 1st stage of mapping (to Ensembl using rs identifiers)

ClinVar_1st = pd.read_csv('intermediate/ClinVar_mapped_to_Ens_1st.csv')
ClinVar_1st

In [ ]:
#how many variants in it

len(ClinVar_1st['id'].unique())

In [ ]:
#list of varinat isd in it

ClinVar_1st_list = ClinVar_1st['id'].unique().tolist()
ClinVar_1st_list

In [ ]:
#ClinVar 2nd stage of mapping (to Ensembl using coordinates)

ClinVar_2nd = pd.read_csv('intermediate/ClinVar_mapped_to_Ens_2nd.csv')
ClinVar_2nd

In [ ]:
##how many variants in it

len(ClinVar_2nd['id'].unique())

In [ ]:
#list of varinat isd in it

ClinVar_2nd_list = ClinVar_2nd['id'].unique().tolist()
ClinVar_2nd_list

In [ ]:
#do they intersect?

venn2(subsets = (set(ClinVar_1st_list), set(ClinVar_2nd_list)), set_labels = ('ClinVar_1st', 'ClinVar_2nd'), 
      set_colors = ("gold", "royalblue"), alpha=1)
plt.show()
#they intersect juct by 8 variants which is good :)
#Save the diagram in high quality
#plt.savefig('/Users/ksenia/Documents/MODY_genes/pipeline_october2022/ClinVar_1st_2nd_intersect.png', dpi=300)

In [ ]:
#Everything from ClinVar mapped to Ensembl

ClinVar_Ens = pd.concat([ClinVar_1st, ClinVar_2nd]).drop_duplicates().reset_index(drop=True)
ClinVar_Ens

In [ ]:
#creating yhr columns for VCF

ClinVar_Ens_VCF = []
for index, row in ClinVar_Ens.iterrows():
    ClinVar_Ens_VCF.append({'chrom': row['coordinate'].replace('>', ':').split(':')[0],
     'pos': row['coordinate'].replace('>', ':').split(':')[1],
     'ref': row['coordinate'].replace('>', ':').split(':')[2],
     'alt': row['coordinate'].replace('>', ':').split(':')[3],
     'accession': row['id']})
ClinVar_Ens_VCF_df = pd.DataFrame(ClinVar_Ens_VCF)
ClinVar_Ens_VCF_df

In [ ]:
len(ClinVar_Ens['id'].unique())

In [ ]:
ClinVar_Ens_list = ClinVar_Ens['coordinate'].unique().tolist()
len(ClinVar_Ens_list)

ClinVar 3rd - fetched SNVs (the variants for which the molecular consequences have been fetched) <- ready to concat

In [ ]:
ClinVar_3rd = pd.read_csv(
    'intermediate/ClinVar_unmapped_filtered_SNV_for_VCF.csv')
ClinVar_3rd

In [ ]:
len(ClinVar_3rd['accession'].unique())

In [ ]:
ClinVar_3rd_list = [] 
for index, row in ClinVar_3rd.iterrows():
    ClinVar_3rd_list.append(str(row['chrom']) + ':' + str(row['pos']) + ':' + row['ref'] + '>' + row['alt']) 
ClinVar_3rd_list

In [ ]:
#ClinVar indels edded unfiltered (actually this contains some SNPs as well, no reason identified) <- ready to concat

ClinVar_indels = pd.read_csv(
    'intermediate/ClinVar_indels.csv')
ClinVar_indels = ClinVar_indels.replace(nan, '')
ClinVar_indels

In [ ]:
len(ClinVar_indels['accession'].unique())

In [ ]:
ClinVar_indels_list = [] 
for index, row in ClinVar_indels.iterrows():
    ClinVar_indels_list.append(str(row['chrom']) + ':' + str(row['pos']) + ':' + row['ref'] + '>' + row['alt']) 
ClinVar_indels_list

In [ ]:
ClinVar_total_set = set(ClinVar_Ens_list + ClinVar_3rd_list + ClinVar_indels_list)
len(ClinVar_total_set)

In [ ]:
Rafique_1st = pd.read_csv(
    'intermediate/Rafique_mapped_to_Ens_1st.csv')
Rafique_1st

In [ ]:
len(Rafique_1st['id'].unique())

In [ ]:
Rafique_2nd = pd.read_csv(
    'intermediate/Rafique_mapped_to_Ens_2nd.csv', low_memory=False)
Rafique_2nd

In [ ]:
len(Rafique_2nd['id'].unique())

In [ ]:
#Everything from Rafique mapped to Ensembl
Rafique_Ens = pd.concat([Rafique_1st, Rafique_2nd]).drop_duplicates().reset_index(drop=True)
Rafique_Ens

In [ ]:
len(Rafique_Ens['id'].unique())

In [ ]:
Rafique_Ens_VCF = []
for index, row in Rafique_Ens.iterrows():
    Rafique_Ens_VCF.append({'chrom': row['coordinate'].replace('>', ':').split(':')[0],
     'pos': row['coordinate'].replace('>', ':').split(':')[1],
     'ref': row['coordinate'].replace('>', ':').split(':')[2],
     'alt': row['coordinate'].replace('>', ':').split(':')[3],
     'accession': row['id']})
Rafique_Ens_VCF_df = pd.DataFrame(Rafique_Ens_VCF)
Rafique_Ens_VCF_df

In [ ]:
Rafique_Ens_list = Rafique_Ens['coordinate'].unique().tolist()
Rafique_Ens_list

In [ ]:
Rafique_manual = pd.read_csv(
    'input/Rafique_manual.csv')
Rafique_manual = Rafique_manual.replace(nan, '')
Rafique_manual

In [ ]:
Rafique_manual_list = [] 
for index, row in Rafique_manual.iterrows():
    Rafique_manual_list.append(str(row['chrom']) + ':' + str(row['pos']) + ':' + row['ref'] + '>' + row['alt']) 
Rafique_manual_list

In [ ]:
Rafique_all_set = set(Rafique_Ens_list + Rafique_manual_list)
len(Rafique_all_set)

In [ ]:
venn2(subsets = (ClinVar_total_set, Rafique_all_set), set_labels = ('ClinVar', 'Rafique'), 
      set_colors = ("gold", "royalblue"), alpha=1)
plt.savefig('figures/ClinVar_Rafique_level1.png', dpi=300)

In [ ]:
VCF = pd.concat([ClinVar_Ens_VCF_df, ClinVar_3rd, ClinVar_indels, Rafique_Ens_VCF_df, Rafique_manual]).drop_duplicates().reset_index(drop=True)
VCF

In [ ]:
VCF.to_csv(
    'intermediate/MD_variants_forVCF_level1.csv',
    header=True, index=False)

## 2. Small tables (ClinVar with just pathogenic + likely pathogenic, Rafique table without 3 canceled genes)

ClinVar 1st stage of mapping pathogenic (to Ensembl using rs identifiers) 'ClinVar_pathogenic_mapped_1st.csv'

ClinVar 2nd stage of mapping pathogenic (to Ensembl using coordinates) 'ClinVar_pathogenic_mapped_to_Ens_2nd.csv'

ClinVar 3rd - fetched SNVs (the variants for which the molecular consequences have been fetched) <- ready to concat 'ClinVar_pat_unmapped_filtered_SNV_for_VCF.csv'

ClinVar indels edded unfiltered (actually this contains some SNPs as well, no reason identified) <- ready to concat 'ClinVar_pathogenic_indels.csv'

Rafique 1st stage of annotatopn (with Vcfanno) 'Rafique_mapped_to_Ens_1st_excluded BLK_KLF11_PAX4.csv'

Rafique 2nd stage and manual are not added as their pathogenicity may be doubtfull 

In [ ]:
#ClinVar 1st stage of mapping (to Ensembl using rs identifiers)
ClinVar_1st_pat = pd.read_csv(
    'intermediate/ClinVar_pathogenic_mapped_1st.csv')

#list of varinat isd in it
ClinVar_1st_pat_list = ClinVar_1st_pat['id'].unique().tolist()

#ClinVar 2nd stage of mapping (to Ensembl using coordinates)
ClinVar_2nd_pat = pd.read_csv(
    'intermediate/ClinVar_pathogenic_mapped_to_Ens_2nd.csv')

#list of varinat isd in it
ClinVar_2nd_pat_list = ClinVar_2nd_pat['id'].unique().tolist()

#Everything from ClinVar mapped to Ensembl
ClinVar_Ens_pat = pd.concat([ClinVar_1st_pat, ClinVar_2nd_pat]).drop_duplicates().reset_index(drop=True)

#creating yhr columns for VCF
ClinVar_Ens_VCF_pat = []
for index, row in ClinVar_Ens_pat.iterrows():
    ClinVar_Ens_VCF_pat.append({'chrom': row['coordinate'].replace('>', ':').split(':')[0],
     'pos': row['coordinate'].replace('>', ':').split(':')[1],
     'ref': row['coordinate'].replace('>', ':').split(':')[2],
     'alt': row['coordinate'].replace('>', ':').split(':')[3],
     'accession': row['id']})
ClinVar_Ens_VCF_df_pat = pd.DataFrame(ClinVar_Ens_VCF_pat)
ClinVar_Ens_pat_list = ClinVar_Ens_pat['coordinate'].unique().tolist()

#ClinVar 3rd - fetched SNVs (the variants for which the molecular consequences have been fetched) <- ready to concat
ClinVar_3rd_pat = pd.read_csv(
    'intermediate/ClinVar_pat_unmapped_filtered_SNV_for_VCF.csv')
ClinVar_3rd_pat_list = [] 
for index, row in ClinVar_3rd_pat.iterrows():
    ClinVar_3rd_pat_list.append(str(row['chrom']) + ':' + str(row['pos']) + ':' + row['ref'] + '>' + row['alt']) 

#ClinVar indels edded unfiltered (actually this contains some SNPs as well, no reason identified) <- ready to concat
ClinVar_indels_pat = pd.read_csv(
    'intermediate/ClinVar_pathogenic_indels.csv')
ClinVar_indels_pat = ClinVar_indels_pat.replace(nan, '')
ClinVar_indels_pat_list = [] 
for index, row in ClinVar_indels_pat.iterrows():
    ClinVar_indels_pat_list.append(str(row['chrom']) + ':' + str(row['pos']) + ':' + row['ref'] + '>' + row['alt'])
ClinVar_total_set_pat = set(ClinVar_Ens_pat_list + ClinVar_3rd_pat_list + ClinVar_indels_pat_list)

#Rafique 1st stage
Rafique_pat = pd.read_csv(
    'intermediate/Rafique_mapped_to_Ens_1st_excluded_BLK_KLF11_PAX4.csv')
Rafique_Ens_VCF_pat = []
for index, row in Rafique_pat.iterrows():
    Rafique_Ens_VCF_pat.append({'chrom': row['coordinate'].replace('>', ':').split(':')[0],
     'pos': row['coordinate'].replace('>', ':').split(':')[1],
     'ref': row['coordinate'].replace('>', ':').split(':')[2],
     'alt': row['coordinate'].replace('>', ':').split(':')[3],
     'accession': row['id']})
Rafique_Ens_VCF_pat_df = pd.DataFrame(Rafique_Ens_VCF_pat)
Rafique_pat_list = Rafique_pat['coordinate'].unique().tolist()
Rafique_pat_set = set(Rafique_pat_list)
VCF_pat = pd.concat([ClinVar_Ens_VCF_df_pat, ClinVar_3rd_pat, ClinVar_indels_pat, Rafique_Ens_VCF_pat_df]).drop_duplicates().reset_index(drop=True)
VCF_pat.to_csv(
    'intermediate/MD_variants_forVCF_level2.csv',
    header=True, index=False)


In [ ]:
VCF_pat

In [ ]:
venn2(subsets = (ClinVar_total_set_pat, Rafique_pat_set), set_labels = ('ClinVar', 'Rafique')
     ,set_colors = ("gold", "royalblue"), alpha=1)
plt.savefig('figures/ClinVar_Rafique_level2.png', dpi=300)

In [ ]:
#Comparisson of the Level1 table to the Level2 table
venn2(subsets = (ClinVar_total_set.union(Rafique_all_set), 
                 ClinVar_total_set_pat.union(Rafique_pat_set)), 
                 set_labels = ('level1', 'level2'), 
                 set_colors = ("gold", "royalblue"), alpha=1)
plt.savefig('figures/level1_level2.png', dpi=300)